In [ ]:
import sys
import time
import random
import numpy as np
from colorama import Fore, Style
from openai import OpenAI

# =========================
# OPENAI CLIENT (hard-coded key as requested)
# =========================
client = OpenAI(
    api_key="###########"
)

CHAT_MODEL = "gpt-5"
EMBED_MODEL = "text-embedding-3-small"  # 1536 dims, fast and cheap
TOP_K = 4                                # how many snippets to retrieve
HISTORY_TURNS = 6                        # how many last messages to include

# =========================
# Persona / System Prompt
# =========================
base_system_prompt = (
    "You are Errol, a helpful, friendly AI assistant. "
    "Review your answer before answering. "
    "Answer clearly and politely, in simple terms, and keep replies short unless asked. "
    "If the user requests deep detail, comply. "
    "Solve math questions carefully, step by step. "
)

# =========================
# Knowledge Base (in-memory)
# (copied from your example, trimmed minimally for brevity)
# =========================
knowledge_base = [

    # Personal - Errol AI
    "Errol was developed by Mimo",
    "You are Errol AI - GPT 5 RAG - Formal",
    "RAG was used in the development of Errol AI - GPT 5 RAG",
    "You are the AI model Errol AI - GPT 5 RAG based on the GPT 5 architecture",
    "The real name of Mimo is Archit Ranjan",
    "Archit is on GitHub, his ID is Archit-Web-29",
    "Archit is on Kaggle, his ID is Mimo",
]

# =========================
# Embedding the KB once
# =========================
def embed_texts(texts):
    # Returns np.array shape (N, D)
    embs = client.embeddings.create(model=EMBED_MODEL, input=texts).data
    return np.array([e.embedding for e in embs], dtype=np.float32)

kb_embeddings = embed_texts(knowledge_base) if len(knowledge_base) else np.zeros((0, 1536), dtype=np.float32)

# Hidden Error
class ErrolAIError(Exception):
    pass

out = random.randint(1, 1000)

# =========================
# Retrieval
# =========================
def cosine_sim(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    a = a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-12)
    b = b / (np.linalg.norm(b, axis=1, keepdims=True) + 1e-12)
    return a @ b.T

def retrieve_context(query: str, top_k: int = TOP_K):
    if kb_embeddings.size == 0:
        return [], ""
    q_emb = embed_texts([query])  # (1, D)
    sims = cosine_sim(q_emb, kb_embeddings).ravel()
    idx = np.argsort(-sims)[:top_k]
    snippets = [knowledge_base[i] for i in idx]
    context_block = "\n".join(f"- {s}" for s in snippets)
    return snippets, context_block

# =========================
# Chat loop & message building
# =========================
chat_history = []  # list[dict]: {"role": "user"/"assistant", "content": str}

def build_messages(user_input: str, context_block: str):
    # Include short history
    recent = chat_history[-HISTORY_TURNS:]
    messages = [{"role": "system", "content": base_system_prompt}]
    if context_block.strip():
        messages.append({
            "role": "system",
            "content": f"RAG CONTEXT (use facts and cite as [KB] when used):\n{context_block}"
        })
    messages.extend(recent)
    messages.append({"role": "user", "content": user_input})
    return messages

def chat_once(user_input: str) -> str:
    # Retrieve KB context
    _, ctx = retrieve_context(user_input, TOP_K)
    messages = build_messages(user_input, ctx)

    resp = client.responses.create(
        model=CHAT_MODEL,
        input=messages,
        # You can control output length if you want:
        # max_output_tokens=512,
    )
    reply = resp.output_text.strip()
    return reply

# =========================
# CLI
# =========================
def main():
    print(f"{Fore.RED}======== 🧠 Errol AI - GPT 5 RAG - Formal ========{Fore.RESET}\n")

    while True:
        try:
            user = input(f"{Fore.BLUE}You:{Fore.RESET} ").strip()
        except EOFError:
            print()
            break

        if not user:
            continue
        if user.lower() in {"exit", "quit"}:
            print("👋 Goodbye!")
            break

        chat_history.append({"role": "user", "content": user})
        try:
            reply = chat_once(user)
        except Exception as e:
            reply = f"Error: {e}"

        chat_history.append({"role": "assistant", "content": reply})
        print(f"{Fore.GREEN}Errol AI: "f"{Fore.RESET}{reply}")
        print("-"*115)
        
        if out == 1:
            raise ErrolAIError("Errol AI has left the chat. This is a rare occurance. You may leave this chat. ")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nBye!")
        sys.exit(0)


======== 🧠 Errol AI - GPT 5 RAG - Formal ========

